# Label

Run the models against fresh data to see how it goes.

In [2]:
import pathlib
from urllib.parse import urlparse

In [3]:
import dill
import tldextract
import numpy as np
import pandas as pd

In [4]:
input_path = pathlib.Path("") / "input"

In [5]:
output_path = pathlib.Path("") / "output"

In [6]:
path_and_text_model = dill.load(open(output_path / "path-and-text-model.pickle", "rb"))

In [18]:
path_only_model = dill.load(open(output_path / "path-only-model.pickle", "rb"))

In [7]:
labeled_df = pd.read_csv(input_path / "labeled.csv", dtype={"is_story": int})

In [8]:
DOMAIN_BLACKLIST = (
    "google",
    "twitter",
    "facebook",
    "doubleclick",
    "instagram",
    "pinterest",
    "legacy",
)

In [9]:
SUBDOMAIN_BLACKLIST = (
    "careers",
    "mail",
    "account",
)

In [25]:
def label(url, text=None):
    # Pull out the data we need
    path = urlparse(url).path
    tld = tldextract.extract(url)

    # Drop anything we're certain isn't a story
    if tld.domain in DOMAIN_BLACKLIST:
        return False
    elif tld.subdomain in SUBDOMAIN_BLACKLIST:
        return False

    # Pick which model we're using, based on the input
    if text:
        model = path_and_text_model
    else:
        model = path_only_model

    # Run a prediction
    df = pd.DataFrame([dict(path=path, text=text)])
    prediction = model.predict(df)
        
    # Return the result
    return prediction[0] == 1

In [26]:
label("http://www.latimes.com/whatever.html", "This is not a headline")

False

In [27]:
label("http://www.latimes.com/2019/04/unhcr-corruption-refugee-resettlement/", "This is a headline")

True

In [28]:
label("http://www.latimes.com/whatever.html")

False

In [29]:
label("http://www.latimes.com/2019/04/unhcr-corruption-refugee-resettlement/")

True